#Command to run local server:

jupyter notebook --no-browser --allow-root --NotebookApp.allow_origin='https://colab.research.google.com' --NotebookApp.token=''  --NotebookApp.disable_check_xsrf=True


the above is the comand line to connect to local host when starting a jupyter notebook (remember to be in the corrent directory)

#UPLOAD DATA:

In [0]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import np_utils
from sklearn.preprocessing import OneHotEncoder

In [32]:
cd /mnt/c/Users/mansi/Desktop/High\ School/senior/Applied\ AI/Loan\ data/lending\ club\ data

/mnt/c/Users/mansi/Desktop/High School/senior/Applied AI/Loan data/lending club data


In [33]:
ls

desktop.ini*                                 LoanStats_LendingClub_2016Q4.csv*
LCDataDictionary.xlsx*                       LoanStats_LendingClub_2017Q1.csv*
LoanStats3a_securev1_LendingClub_07-11.csv*  LoanStats_LendingClub_2017Q2.csv*
LoanStats3b_securev1_LendingClub_12-13.csv*  LoanStats_LendingClub_2017Q3.csv*
LoanStats3c_LendingClub_2014.csv*            LoanStats_LendingClub_2017Q4.csv*
LoanStats3d_LendingClub_2015.csv*            LoanStats_LendingClub_2018Q1.csv*
LoanStats_LendingClub_2016Q1.csv*            LoanStats_LendingClub_2018Q2.csv*
LoanStats_LendingClub_2016Q2.csv*            LoanStats_LendingClub_2018Q3.csv*
LoanStats_LendingClub_2016Q3.csv*            RejectStatsA.csv*


In [34]:
#for host run time
'''from google.colab import drive
drive.mount('/content/drive')'''

"from google.colab import drive\ndrive.mount('/content/drive')"

In [35]:
#for host run time
'''#!pwd
%cd drive/My\ Drive
%cd Colab\ Notebooks
%cd Loan\ Project
# we are in the "Loan Project directory" - if there is an issue use "%cd ... to reenter directory"
#!ls
#!cat LoanStats3a_securev1_LendingClub_07-11.csv'''

'#!pwd\n%cd drive/My\\ Drive\n%cd Colab\\ Notebooks\n%cd Loan\\ Project\n# we are in the "Loan Project directory" - if there is an issue use "%cd ... to reenter directory"\n#!ls\n#!cat LoanStats3a_securev1_LendingClub_07-11.csv'

In [36]:
#data is from Lending Club - I have renamed csv files
df_07_11 = pd.read_csv('LoanStats3a_securev1_LendingClub_07-11.csv', sep=",", skiprows=1)
df_12_13 = pd.read_csv('LoanStats3b_securev1_LendingClub_12-13.csv', sep=",", skiprows=1)
df_14 = pd.read_csv("LoanStats3c_LendingClub_2014.csv", sep=",", skiprows=1)
df_15 = pd.read_csv('LoanStats3d_LendingClub_2015.csv', sep=",", skiprows=1)
df_16Q1 = pd.read_csv('LoanStats_LendingClub_2016Q1.csv', sep=",", skiprows=1)
df_16Q2 = pd.read_csv('LoanStats_LendingClub_2016Q2.csv', sep=",", skiprows=1)
df_16Q3 = pd.read_csv('LoanStats_LendingClub_2016Q3.csv', sep=",", skiprows=1)
df_16Q4 = pd.read_csv('LoanStats_LendingClub_2016Q4.csv', sep=",", skiprows=1)
df_17Q1 = pd.read_csv('LoanStats_LendingClub_2017Q1.csv', sep=",", skiprows=1)
df_17Q2 = pd.read_csv('LoanStats_LendingClub_2017Q2.csv', sep=",", skiprows=1)
df_17Q3 = pd.read_csv('LoanStats_LendingClub_2017Q3.csv', sep=",", skiprows=1)
df_17Q4 = pd.read_csv('LoanStats_LendingClub_2017Q4.csv', sep=",", skiprows=1)
df_18Q1 = pd.read_csv('LoanStats_LendingClub_2018Q1.csv', sep=",", skiprows=1)
df_18Q2 = pd.read_csv('LoanStats_LendingClub_2018Q2.csv', sep=",", skiprows=1)
df_18Q3 = pd.read_csv('LoanStats_LendingClub_2018Q3.csv', sep=",", skiprows=1)

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,49,129,130,131,134,135,136,139) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, 

#PROCESS THE DATA:

In [37]:
frames = [df_07_11, df_12_13, df_14, df_15, df_16Q1, df_16Q2, df_16Q3, df_16Q4, df_17Q1, df_17Q2, df_17Q3, df_17Q4, df_18Q1,df_18Q2,df_18Q3]

result = pd.concat(frames, ignore_index=True)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [38]:
#shuffle enties - if you use dates at some point, dont shuffle
result = result.reindex(np.random.permutation(result.index)) #for now we arn't using data, so we shuffle
result.shape

(2132289, 151)

In [39]:
result.drop(columns=['id', 'member_id','url','zip_code', 'issue_d'], inplace=True)
result.shape

(2132289, 146)

In [40]:
'''
#"loan_status" is the column heading of whether it defulted or not
x = result.copy()
#labels = df.loc[:,"loan_status"]
y = result['loan_status'].copy()
y.shape[0]
'''
#the above code is implemented later, and thus is useless

2132289

In [41]:
label = []
for i in range(y.shape[0]):
  if y[i] not in label:
    label.append(y[i])

for i in range(len(label)):
  print(label[i])
  
print("There are", len(label), "labels")

Fully Paid
Charged Off
nan
Does not meet the credit policy. Status:Fully Paid
Does not meet the credit policy. Status:Charged Off
Late (31-120 days)
Current
Late (16-30 days)
In Grace Period
Default
There are 10 labels


##NOTES about loan_status (aka. labels):


*   if label is "nan" - we throw that loan out of the training set
*   Current means that the loan is still being paid off, and therefore we cannot use for out training - might be useful to comeback and revisit in the future (maybe predict outcome now) - will be thrown out of training set
*   fully paid  - successful loan


---


*   what is "charged off" vs "default"
 *   "Loans that are in "Default" are loans for which borrowers have failed to make payments for an extended period of time. A loan becomes “Charged Off” when there is no longer a reasonable expectation of further payments.  Charge Off typically occurs when a loan is 120 days or more past due and there is no reasonable expectation of sufficient payment to prevent the charge off.  In certain circumstances, loans may be charged off at an earlier or later date.  Please note, loans for which borrowers have filed for bankruptcy may be charged off earlier based on the date of bankruptcy notification.
 A loan that is in “Default” will still appear in your Notes, in the status of “Default,” while a loan that has been “Charged Off” will appear as charged off, and the remaining principal balance of the Note will be deducted from your account balance.  Learn more about what happens when a loan is charged off"
 *   Because in the case of "charged off" vs "default" the lender has extremely low chance of recovering their investment, we will treat both of these classifications as if they are the same thing

---


*   what does it mean if it "does not meet the credit policy"
 *   Lending Club doesn't make clear the meaning of this statement, but from general internet research most ppl just ommit the "does not meet the credit policy" part and use the second half of the classification
---
*   what does "grace period" mean - should we potentially throw those away?
 * Loan is past due but within the 15-day grace period. - we will throw these away because for training data it isn't useful since the final status of loan isn't avaliable
*   if its late - does that mean status is still being finalized? - should we throw away?
 * if it is late we don't use this loan in training set cus finalized status isn't avalible to confirm predictions



In [42]:
result = result[~result['loan_status'].isin(['Late (31-120 days)', 'Current', 'Late (16-30 days)', 'In Grace Period'])]
# above gets rid of rows that have loan status' that we don't want to consider
result = result[~result['loan_status'].isna()] 

#make labels binary
pay_status = {'Fully Paid':1, 'Charged Off':0, 'Does not meet the credit policy. Status:Fully Paid':1, 'Does not meet the credit policy. Status:Charged Off':0, 'Default':0}
result.loan_status = [pay_status[item] for item in result.loan_status]


# above gets rid of rows that don't have a valid loan status
result.shape

(1201050, 146)

In [43]:
#DROPPING instances of larger ("paid") class - didn't show much effect
#result = result.drop(result[result.loan_status.eq(1)].sample(frac=.75).index)
result.shape

(1201050, 146)

In [44]:
#"loan_status" is the column heading of whether it defulted or not
x = result.copy()


#labels = df.loc[:,"loan_status"]
y = result['loan_status'].copy()
#get rid of labels from test set
x.drop(columns=['loan_status'], inplace=True) 

#for now, we are gonna drop all columns that have even one NAN value - this signifigantly reduces training data (so its something we want to comeback and deal with later)
x.dropna(axis='columns', inplace=True)
print(list(x))

#y #can print out labels
print(x.shape)
x

['addr_state', 'application_type', 'collection_recovery_fee', 'debt_settlement_flag', 'disbursement_method', 'funded_amnt', 'funded_amnt_inv', 'grade', 'hardship_flag', 'home_ownership', 'initial_list_status', 'installment', 'int_rate', 'last_pymnt_amnt', 'loan_amnt', 'out_prncp', 'out_prncp_inv', 'policy_code', 'purpose', 'pymnt_plan', 'recoveries', 'revol_bal', 'sub_grade', 'term', 'total_pymnt', 'total_pymnt_inv', 'total_rec_int', 'total_rec_late_fee', 'total_rec_prncp', 'verification_status']
(1201050, 30)


,addr_state,application_type,collection_recovery_fee,debt_settlement_flag,disbursement_method,funded_amnt,funded_amnt_inv,grade,hardship_flag,home_ownership,...,recoveries,revol_bal,sub_grade,term,total_pymnt,total_pymnt_inv,total_rec_int,total_rec_late_fee,total_rec_prncp,verification_status
783788,TN,Individual,1440.0000,Y,Cash,13600.0,13600.00000,E,N,MORTGAGE,...,8000.00,7278.0,E3,60 months,12964.300000,12964.30,2853.49,0.00,2110.81,Source Verified
661671,WA,Individual,0.0000,N,Cash,4000.0,4000.00000,C,N,RENT,...,0.00,12333.0,C4,36 months,4444.920000,4444.92,444.92,0.00,4000.00,Not Verified
1293389,MO,Individual,0.0000,N,Cash,2975.0,2975.00000,D,N,MORTGAGE,...,0.00,44512.0,D5,36 months,1234.140000,1234.14,517.02,0.00,717.12,Source Verified
717904,TX,Individual,0.0000,N,Cash,9600.0,9600.00000,D,N,RENT,...,0.00,18367.0,D1,36 months,12171.123819,12171.12,2554.34,16.78,9600.00,Verified
516267,AL,Individual,0.0000,N,Cash,2500.0,2500.00000,B,N,MORTGAGE,...,0.00,6024.0,B2,36 months,2853.533742,2853.53,353.53,0.00,2500.00,Verified
1501243,PA,Individual,0.0000,N,Cash,15000.0,15000.00000,B,N,MORTGAGE,...,0.00,16881.0,B1,36 months,15841.338803,15841.34,841.34,0.00,15000.00,Not Verified
561684,MO,Individual,0.0000,N,Cash,15000.0,15000.00000,A,N,RENT,...,0.00,7449.0,A1,36 months,16203.118382,16203.12,1203.12,0.00,15000.00,Not Verified
730882,WI,Individual,209.7198,N,Cash,12000.0,12000.00000,B,N,MORTGAGE,...,1165.11,17937.0,B1,36 months,3424.620000,3424.62,457.61,0.00,1801.90,Not Verified
1202807,CA,Individual,655.0038,Y,Cash,33600.0,33350.00000,F,N,RENT,...,3638.91,5591.0,F2,60 months,10650.400000,10571.16,4961.32,0.00,2050.17,Source Verified
1956927,IL,Joint App,0.0000,N,Cash,30000.0,30000.00000,B,N,MORTGAGE,...,0.00,9558.0,B2,36 months,30026.043333,30026.04,26.04,0.00,30000.00,Source Verified


In [0]:
x['term'] = x['term'].map(lambda x: x.lstrip('%').rstrip('months'))
x['int_rate'] = x["int_rate"].map(lambda x: x.rstrip("%"))
int_rate = x['int_rate'].str.replace(' ', '')
#x['revol_util'] = x['revol_util'].str.replace('%', '') #format changes bc column as floats and ints, also has NaN values, so it was removed above
#print(x["revol_util"])
x.drop(columns=['int_rate'], inplace=True)

In [46]:
int_rate

783788     19.52
661671     13.99
1293389    21.49
717904     15.61
516267      9.17
1501243     8.24
561684      5.32
730882      8.18
1202807    26.49
1956927     9.92
1197470    12.79
107937      9.71
533625     10.99
782516     19.52
1621319    10.91
898029     13.67
845279     12.39
534152      5.32
59707      11.99
962872     16.29
1240991    16.99
2001134     7.96
769446      7.89
489132      9.76
1286679    10.49
21714       6.91
563518      6.89
359562     13.98
1077454    25.44
563222     13.99
           ...  
1489468    30.84
1049053     7.89
1719377    15.05
291711     13.98
2080860    17.97
1670459    18.06
721094     11.53
320822     13.35
1730876     7.97
126491     12.12
130663      6.03
309567     12.49
918680      5.32
512042      9.99
1383115     5.32
1233544    15.99
157548     12.12
1405526    21.49
1104932     8.39
434666     18.25
631054     15.61
691358      6.24
35302      11.48
560145     13.33
540028     11.49
570677     10.99
651641     19.19
573603     10.

In [47]:
#print(x['int_rate'])
x

,addr_state,application_type,collection_recovery_fee,debt_settlement_flag,disbursement_method,funded_amnt,funded_amnt_inv,grade,hardship_flag,home_ownership,...,recoveries,revol_bal,sub_grade,term,total_pymnt,total_pymnt_inv,total_rec_int,total_rec_late_fee,total_rec_prncp,verification_status
783788,TN,Individual,1440.0000,Y,Cash,13600.0,13600.00000,E,N,MORTGAGE,...,8000.00,7278.0,E3,60,12964.300000,12964.30,2853.49,0.00,2110.81,Source Verified
661671,WA,Individual,0.0000,N,Cash,4000.0,4000.00000,C,N,RENT,...,0.00,12333.0,C4,36,4444.920000,4444.92,444.92,0.00,4000.00,Not Verified
1293389,MO,Individual,0.0000,N,Cash,2975.0,2975.00000,D,N,MORTGAGE,...,0.00,44512.0,D5,36,1234.140000,1234.14,517.02,0.00,717.12,Source Verified
717904,TX,Individual,0.0000,N,Cash,9600.0,9600.00000,D,N,RENT,...,0.00,18367.0,D1,36,12171.123819,12171.12,2554.34,16.78,9600.00,Verified
516267,AL,Individual,0.0000,N,Cash,2500.0,2500.00000,B,N,MORTGAGE,...,0.00,6024.0,B2,36,2853.533742,2853.53,353.53,0.00,2500.00,Verified
1501243,PA,Individual,0.0000,N,Cash,15000.0,15000.00000,B,N,MORTGAGE,...,0.00,16881.0,B1,36,15841.338803,15841.34,841.34,0.00,15000.00,Not Verified
561684,MO,Individual,0.0000,N,Cash,15000.0,15000.00000,A,N,RENT,...,0.00,7449.0,A1,36,16203.118382,16203.12,1203.12,0.00,15000.00,Not Verified
730882,WI,Individual,209.7198,N,Cash,12000.0,12000.00000,B,N,MORTGAGE,...,1165.11,17937.0,B1,36,3424.620000,3424.62,457.61,0.00,1801.90,Not Verified
1202807,CA,Individual,655.0038,Y,Cash,33600.0,33350.00000,F,N,RENT,...,3638.91,5591.0,F2,60,10650.400000,10571.16,4961.32,0.00,2050.17,Source Verified
1956927,IL,Joint App,0.0000,N,Cash,30000.0,30000.00000,B,N,MORTGAGE,...,0.00,9558.0,B2,36,30026.043333,30026.04,26.04,0.00,30000.00,Source Verified


In [0]:
#y = np_utils.to_categorical(y)
#Convert categorical variable into dummy/indicator variables:
x =pd.get_dummies(x, drop_first=True)

In [49]:
#print(y[13])
print(list(x))
print(x.shape)
y.shape

['collection_recovery_fee', 'funded_amnt', 'funded_amnt_inv', 'installment', 'last_pymnt_amnt', 'loan_amnt', 'out_prncp', 'out_prncp_inv', 'policy_code', 'recoveries', 'revol_bal', 'total_pymnt', 'total_pymnt_inv', 'total_rec_int', 'total_rec_late_fee', 'total_rec_prncp', 'addr_state_AL', 'addr_state_AR', 'addr_state_AZ', 'addr_state_CA', 'addr_state_CO', 'addr_state_CT', 'addr_state_DC', 'addr_state_DE', 'addr_state_FL', 'addr_state_GA', 'addr_state_HI', 'addr_state_IA', 'addr_state_ID', 'addr_state_IL', 'addr_state_IN', 'addr_state_KS', 'addr_state_KY', 'addr_state_LA', 'addr_state_MA', 'addr_state_MD', 'addr_state_ME', 'addr_state_MI', 'addr_state_MN', 'addr_state_MO', 'addr_state_MS', 'addr_state_MT', 'addr_state_NC', 'addr_state_ND', 'addr_state_NE', 'addr_state_NH', 'addr_state_NJ', 'addr_state_NM', 'addr_state_NV', 'addr_state_NY', 'addr_state_OH', 'addr_state_OK', 'addr_state_OR', 'addr_state_PA', 'addr_state_RI', 'addr_state_SC', 'addr_state_SD', 'addr_state_TN', 'addr_state_T

(1201050,)

In [0]:
#x.add(int_rate, axis='columns', level=None, fill_value=None)
x = pd.concat([x, int_rate], axis=1, join_axes=[x.index])

#dropping non numerical columns for non - will/should come back a one hot encode categorical columns
#x = x.select_dtypes(['number'])
#x['grade']

In [51]:
print(list(x))
print(x.shape)
x

['collection_recovery_fee', 'funded_amnt', 'funded_amnt_inv', 'installment', 'last_pymnt_amnt', 'loan_amnt', 'out_prncp', 'out_prncp_inv', 'policy_code', 'recoveries', 'revol_bal', 'total_pymnt', 'total_pymnt_inv', 'total_rec_int', 'total_rec_late_fee', 'total_rec_prncp', 'addr_state_AL', 'addr_state_AR', 'addr_state_AZ', 'addr_state_CA', 'addr_state_CO', 'addr_state_CT', 'addr_state_DC', 'addr_state_DE', 'addr_state_FL', 'addr_state_GA', 'addr_state_HI', 'addr_state_IA', 'addr_state_ID', 'addr_state_IL', 'addr_state_IN', 'addr_state_KS', 'addr_state_KY', 'addr_state_LA', 'addr_state_MA', 'addr_state_MD', 'addr_state_ME', 'addr_state_MI', 'addr_state_MN', 'addr_state_MO', 'addr_state_MS', 'addr_state_MT', 'addr_state_NC', 'addr_state_ND', 'addr_state_NE', 'addr_state_NH', 'addr_state_NJ', 'addr_state_NM', 'addr_state_NV', 'addr_state_NY', 'addr_state_OH', 'addr_state_OK', 'addr_state_OR', 'addr_state_PA', 'addr_state_RI', 'addr_state_SC', 'addr_state_SD', 'addr_state_TN', 'addr_state_T

,collection_recovery_fee,funded_amnt,funded_amnt_inv,installment,last_pymnt_amnt,loan_amnt,out_prncp,out_prncp_inv,policy_code,recoveries,...,sub_grade_F5,sub_grade_G1,sub_grade_G2,sub_grade_G3,sub_grade_G4,sub_grade_G5,term_ 60,verification_status_Source Verified,verification_status_Verified,int_rate
783788,1440.0000,13600.0,13600.00000,356.70,356.70,13600.0,0.0,0.0,1.0,8000.00,...,0,0,0,0,0,0,1,1,0,19.52
661671,0.0000,4000.0,4000.00000,136.70,3197.97,4000.0,0.0,0.0,1.0,0.00,...,0,0,0,0,0,0,0,0,0,13.99
1293389,0.0000,2975.0,2975.00000,112.84,112.84,2975.0,0.0,0.0,1.0,0.00,...,0,0,0,0,0,0,0,1,0,21.49
717904,0.0000,9600.0,9600.00000,335.67,383.92,9600.0,0.0,0.0,1.0,0.00,...,0,0,0,0,0,0,0,0,1,15.61
516267,0.0000,2500.0,2500.00000,79.70,469.61,2500.0,0.0,0.0,1.0,0.00,...,0,0,0,0,0,0,0,0,1,9.17
1501243,0.0000,15000.0,15000.00000,471.71,12087.22,15000.0,0.0,0.0,1.0,0.00,...,0,0,0,0,0,0,0,0,0,8.24
561684,0.0000,15000.0,15000.00000,451.73,3559.11,15000.0,0.0,0.0,1.0,0.00,...,0,0,0,0,0,0,0,0,0,5.32
730882,209.7198,12000.0,12000.00000,377.04,377.04,12000.0,0.0,0.0,1.0,1165.11,...,0,0,0,0,0,0,0,0,0,8.18
1202807,655.0038,33600.0,33350.00000,1015.77,1015.77,33600.0,0.0,0.0,1.0,3638.91,...,0,0,0,0,0,0,1,1,0,26.49
1956927,0.0000,30000.0,30000.00000,966.89,30042.57,30000.0,0.0,0.0,1.0,0.00,...,0,0,0,0,0,0,0,1,0,9.92


In [76]:
y

783788     0
661671     1
1293389    0
717904     1
516267     1
1501243    1
561684     1
730882     0
1202807    0
1956927    1
1197470    1
107937     1
533625     1
782516     1
1621319    1
898029     1
845279     1
534152     1
59707      1
962872     0
1240991    0
2001134    1
769446     1
489132     1
1286679    0
21714      1
563518     1
359562     1
1077454    1
563222     1
          ..
1489468    1
1049053    1
1719377    0
291711     1
2080860    1
1670459    1
721094     1
320822     1
1730876    1
126491     1
130663     1
309567     1
918680     1
512042     1
1383115    1
1233544    0
157548     1
1405526    1
1104932    1
434666     1
631054     1
691358     1
35302      1
560145     1
540028     1
570677     0
651641     0
573603     1
949973     0
606141     0
Name: loan_status, Length: 1201050, dtype: int64

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

seed = 45
np.random.seed(seed)

In [86]:
x

,collection_recovery_fee,funded_amnt,funded_amnt_inv,installment,last_pymnt_amnt,loan_amnt,out_prncp,out_prncp_inv,policy_code,recoveries,...,sub_grade_F5,sub_grade_G1,sub_grade_G2,sub_grade_G3,sub_grade_G4,sub_grade_G5,term_ 60,verification_status_Source Verified,verification_status_Verified,int_rate
783788,1440.0000,13600.0,13600.00000,356.70,356.70,13600.0,0.0,0.0,1.0,8000.00,...,0,0,0,0,0,0,1,1,0,19.52
661671,0.0000,4000.0,4000.00000,136.70,3197.97,4000.0,0.0,0.0,1.0,0.00,...,0,0,0,0,0,0,0,0,0,13.99
1293389,0.0000,2975.0,2975.00000,112.84,112.84,2975.0,0.0,0.0,1.0,0.00,...,0,0,0,0,0,0,0,1,0,21.49
717904,0.0000,9600.0,9600.00000,335.67,383.92,9600.0,0.0,0.0,1.0,0.00,...,0,0,0,0,0,0,0,0,1,15.61
516267,0.0000,2500.0,2500.00000,79.70,469.61,2500.0,0.0,0.0,1.0,0.00,...,0,0,0,0,0,0,0,0,1,9.17
1501243,0.0000,15000.0,15000.00000,471.71,12087.22,15000.0,0.0,0.0,1.0,0.00,...,0,0,0,0,0,0,0,0,0,8.24
561684,0.0000,15000.0,15000.00000,451.73,3559.11,15000.0,0.0,0.0,1.0,0.00,...,0,0,0,0,0,0,0,0,0,5.32
730882,209.7198,12000.0,12000.00000,377.04,377.04,12000.0,0.0,0.0,1.0,1165.11,...,0,0,0,0,0,0,0,0,0,8.18
1202807,655.0038,33600.0,33350.00000,1015.77,1015.77,33600.0,0.0,0.0,1.0,3638.91,...,0,0,0,0,0,0,1,1,0,26.49
1956927,0.0000,30000.0,30000.00000,966.89,30042.57,30000.0,0.0,0.0,1.0,0.00,...,0,0,0,0,0,0,0,1,0,9.92


In [53]:
print(X_train.size+X_test.size)

160940700


In [54]:
print(y_train.size + y_test.size)
X_train

1201050


,collection_recovery_fee,funded_amnt,funded_amnt_inv,installment,last_pymnt_amnt,loan_amnt,out_prncp,out_prncp_inv,policy_code,recoveries,...,sub_grade_F5,sub_grade_G1,sub_grade_G2,sub_grade_G3,sub_grade_G4,sub_grade_G5,term_ 60,verification_status_Source Verified,verification_status_Verified,int_rate
127090,0.0000,15000.0,15000.0,513.33,3489.88,15000.0,0.0,0.0,1.0,0.00,...,0,0,0,0,0,0,0,0,0,14.09
1004652,0.0000,8000.0,8000.0,245.16,5318.02,8000.0,0.0,0.0,1.0,0.00,...,0,0,0,0,0,0,0,0,1,6.49
1490735,0.0000,30000.0,30000.0,678.61,26965.46,30000.0,0.0,0.0,1.0,0.00,...,0,0,0,0,0,0,1,0,0,12.74
117970,0.0000,9500.0,9500.0,326.22,1.11,9500.0,0.0,0.0,1.0,0.00,...,0,0,0,0,0,0,0,0,0,14.33
588584,0.0000,5875.0,5875.0,209.44,209.00,5875.0,0.0,0.0,1.0,0.00,...,0,0,0,0,0,0,0,0,1,16.99
702629,0.0000,13600.0,13600.0,425.49,425.28,13600.0,0.0,0.0,1.0,0.00,...,0,0,0,0,0,0,0,0,1,7.89
1496941,0.0000,7000.0,7000.0,242.63,1853.47,7000.0,0.0,0.0,1.0,0.00,...,0,0,0,0,0,0,0,1,0,14.99
1587474,0.0000,1000.0,1000.0,34.69,913.67,1000.0,0.0,0.0,1.0,0.00,...,0,0,0,0,0,0,0,0,0,15.05
343755,282.4740,29575.0,29575.0,882.52,882.52,29575.0,0.0,0.0,1.0,1569.30,...,0,0,1,0,0,0,1,1,0,25.83
632473,0.0000,24000.0,24000.0,558.32,48.40,24000.0,0.0,0.0,1.0,0.00,...,0,0,0,0,0,0,1,1,0,13.99


# SHALLOW NN:

In [0]:

my_model = Sequential()
my_model.add(Dense(150, input_dim=134, activation='sigmoid'))
my_model.add(Dense(150,  activation='sigmoid'))
my_model.add(Dense(150,  activation='sigmoid'))
my_model.add(Dense(150,  activation='sigmoid'))
my_model.add(Dense(1, activation='softmax')) #when i one hot encode the answer key - u need output nueron number to be 2 not 1

my_model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy']) #need to one hot encode the answer key cus im a dumb dumb

In [56]:
scores = my_model.evaluate(X_train, y_train, verbose=0)
print("Accuracy on training data: %.2f%%" % (scores[1]*100))

scores = my_model.evaluate(X_test, y_test, verbose=0)
print("Accuracy on test data: %.2f%%" % (scores[1]*100))

y_pred = my_model.predict(X_train)

from sklearn.metrics import confusion_matrix
confusion_matrix(y_train, y_pred)

#model is never predicting that the person will not pay

Accuracy on training data: 79.93%
Accuracy on test data: 79.93%


array([[     0, 161481],
       [     0, 643222]])

#BASIC CNN:

In [57]:
x_conv = np.expand_dims(x, axis=2) #need to reshape for CONV ONLY
print(x_conv.shape)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_conv, y, test_size=0.33, random_state=42)

seed = 45
np.random.seed(seed)

(1201050, 134, 1)


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten
#create model
CNNmodel = Sequential()
#add model layers
CNNmodel.add(Conv1D(64, kernel_size=2, activation='relu', input_shape=(134,1)))
CNNmodel.add(Conv1D(32, kernel_size=2, activation='relu'))
CNNmodel.add(Flatten())
CNNmodel.add(Dense(1, activation='softmax'))

CNNmodel.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [59]:
scores = CNNmodel.evaluate(X_train, y_train, verbose=0)
print("Accuracy on training data: %.2f%%" % (scores[1]*100))

scores = CNNmodel.evaluate(X_test, y_test, verbose=0)
print("Accuracy on test data: %.2f%%" % (scores[1]*100))

y_predCNN = CNNmodel.predict(X_train)
confusion_matrix(y_train, y_predCNN)

Accuracy on training data: 79.93%
Accuracy on test data: 79.93%


array([[     0, 161481],
       [     0, 643222]])

#Naive Bayes Classifier
##Gaussian Distribution
###Continuous variables


*   We can also test multinomial and bernoulli models from scikit-learn just to see how they perform for kicks and giggles
*   https://www.youtube.com/watch?v=r1in0YNetG8
  *  the above is a video for gaussian naive bayes
  * use pdf (probability density function) (can google the equation)



In [117]:
x_Naive = pd.concat([x, y], axis=1)
x_Naive = x_Naive.drop(x_Naive[x_Naive.loan_status.eq(1)].sample(frac=.80).index)
x_Naive = x_Naive.drop(x_Naive.sample(frac=.90).index)
y_Naive = x_Naive['loan_status'].copy()
x_Naive.drop(columns=['loan_status'], inplace=True) 
x_Naive.shape

(43304, 134)

In [0]:
#need a function to find class_size
def class_sizeFinder(df, Class):
  return(df[Class].size)

#need function to determine mean of each class
#for Class you need to pass in a df indexed into the class in a way that can be indexed into each individual feature
def mean(df, Class):
  return(df[Class].mean())
  

#need function to determine variance of each class
def variance(df, Class):
  return(df[Class].var())

#need probability density function

In [121]:
colLen = class_sizeFinder(x_Naive,'funded_amnt')
colLen

mean(x_Naive, "funded_amnt")

14859.01475614262

In [123]:
variance(x_Naive, "funded_amnt")

74768687.26225679